# Clustering Analysis

This notebook applies various clustering algorithms to group similar Suricata rules.

We will:
1. Load the feature matrix
2. Apply K-Means clustering
3. Apply DBSCAN clustering
4. Apply Hierarchical clustering
5. Compare algorithm performance
6. Analyze cluster characteristics

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from suricata_rule_clustering import parser, clustering, features
from suricata_rule_clustering.cluster_analysis import ClusterDescriptor, format_cluster_description

# Set display options
pd.set_option('display.max_columns', None)

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Load Data

In [ ]:
# Load feature matrix and processed DataFrame
X = np.load('../data/feature_matrix.npy')
df = pd.read_pickle('../data/processed_rules.pkl')

print(f"Feature matrix shape: {X.shape}")
print(f"DataFrame shape: {df.shape}")

## 2. K-Means Clustering

First, let's find the optimal number of clusters using the elbow method and silhouette score.

In [ ]:
# Find optimal k using elbow method
optimal_k_elbow, elbow_scores = clustering.find_optimal_k(
    X,
    k_range=range(2, 15),
    method='elbow'
)

# Plot elbow curve
plt.figure(figsize=(10, 5))
plt.plot(list(elbow_scores.keys()), list(elbow_scores.values()), 'bo-')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Method For Optimal k')
plt.grid(True)
plt.show()

print(f"Optimal k (elbow): {optimal_k_elbow}")

In [ ]:
# Find optimal k using silhouette score
optimal_k_silhouette, silhouette_scores = clustering.find_optimal_k(
    X,
    k_range=range(2, 15),
    method='silhouette'
)

# Plot silhouette scores
plt.figure(figsize=(10, 5))
plt.plot(list(silhouette_scores.keys()), list(silhouette_scores.values()), 'go-')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score For Different k Values')
plt.grid(True)
plt.show()

print(f"Optimal k (silhouette): {optimal_k_silhouette}")

In [ ]:
# Apply K-Means with chosen k
k = optimal_k_silhouette

kmeans_clusterer = clustering.RuleClusterer(
    algorithm='kmeans',
    n_clusters=k
)
kmeans_clusterer.fit(X)

print(f"K-Means clustering completed with {k} clusters")

In [ ]:
# Evaluate K-Means clustering
kmeans_metrics = clustering.evaluate_clustering(X, kmeans_clusterer.labels_)
print("K-Means Evaluation Metrics:")
for metric, value in kmeans_metrics.items():
    print(f"  {metric}: {value}")

In [ ]:
# Get cluster summary
kmeans_summary = kmeans_clusterer.get_cluster_summary(df)
print("\nK-Means Cluster Summary:")
kmeans_summary.head()

## 3. DBSCAN Clustering

DBSCAN is a density-based algorithm that can find clusters of arbitrary shapes and automatically detect outliers.

In [ ]:
# Apply DBSCAN
# You may need to experiment with eps and min_samples parameters
dbscan_clusterer = clustering.RuleClusterer(
    algorithm='dbscan',
    eps=0.5,
    min_samples=5
)
dbscan_clusterer.fit(X)

print("DBSCAN clustering completed")

In [ ]:
# Evaluate DBSCAN clustering
dbscan_metrics = clustering.evaluate_clustering(X, dbscan_clusterer.labels_)
print("DBSCAN Evaluation Metrics:")
for metric, value in dbscan_metrics.items():
    print(f"  {metric}: {value}")

In [ ]:
# Get cluster summary
dbscan_summary = dbscan_clusterer.get_cluster_summary(df)
print("\nDBSCAN Cluster Summary:")
dbscan_summary

## 4. Hierarchical Clustering

Takes a lot of time, stopped at 40m runtime and disabled for now.

In [ ]:
# # Apply Hierarchical clustering
# hierarchical_clusterer = clustering.RuleClusterer(
#     algorithm='hierarchical',
#     n_clusters=k,  # Use same k as K-Means for comparison
#     linkage='ward'
# )
# hierarchical_clusterer.fit(X)
#
# print("Hierarchical clustering completed")

In [ ]:
# # Evaluate Hierarchical clustering
# hierarchical_metrics = clustering.evaluate_clustering(X, hierarchical_clusterer.labels_)
# print("Hierarchical Clustering Evaluation Metrics:")
# for metric, value in hierarchical_metrics.items():
#     print(f"  {metric}: {value}")

In [ ]:
# # Get cluster summary
# hierarchical_summary = hierarchical_clusterer.get_cluster_summary(df)
# print("\nHierarchical Cluster Summary:")
# hierarchical_summary

In [ ]:
# # Plot dendrogram (with sampled data for performance)
# fig = clustering.plot_dendrogram(X, max_samples=500)
# plt.show()

## 5. Compare Algorithms

This section compares different clustering algorithms to find the best approach for our Suricata rules.

**Performance Optimizations:**
- **Dataset Sampling**: Uses a maximum of 10,000 rules (or 20% of dataset) for faster comparison while maintaining statistical validity
- **PCA Dimensionality Reduction**: Reduces feature dimensions from ~140 to 50, speeding up all distance computations
- **Mini-Batch K-Means**: Replaces slow Hierarchical clustering (40+ min runtime) with faster Mini-Batch K-Means alternative

**Expected Runtime**: 2-5 minutes (down from 40+ minutes)

In [ ]:
import time

# Compare different algorithms and parameters
# Optimizations applied:
# - Dataset sampling: max 10,000 rules for faster comparison
# - PCA dimensionality reduction: ~140 dims → 50 dims
# - Mini-Batch K-Means replaces Hierarchical clustering
print(f"Starting algorithm comparison...")
print(f"Original dataset: {X.shape[0]} samples, {X.shape[1]} features")

start_time = time.time()
comparison = clustering.compare_algorithms(
    X, df,
    max_samples=10000,  # Limit to 10k samples for speed
    apply_pca_reduction=True,  # Apply PCA
    pca_components=50  # Reduce to 50 dimensions
)
elapsed_time = time.time() - start_time

print(f"\nAlgorithm comparison completed in {elapsed_time:.2f} seconds ({elapsed_time/60:.2f} minutes)")
print("\nAlgorithm Comparison:")
comparison

In [ ]:
# Visualize comparison
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot silhouette scores
valid_silhouette = comparison.dropna(subset=['silhouette_score'])
if not valid_silhouette.empty:
    axes[0].barh(valid_silhouette['algorithm'], valid_silhouette['silhouette_score'])
    axes[0].set_xlabel('Silhouette Score')
    axes[0].set_title('Silhouette Score Comparison (Higher is Better)')

# Plot number of clusters
axes[1].barh(comparison['algorithm'], comparison['n_clusters'])
axes[1].set_xlabel('Number of Clusters')
axes[1].set_title('Number of Clusters Found')

plt.tight_layout()
plt.show()

## 6. Analyze Best Clustering Result

Choose the best performing algorithm and analyze its clusters in detail.

In [ ]:
# Select the best clusterer (adjust based on comparison above)
best_clusterer = kmeans_clusterer  # or dbscan_clusterer or hierarchical_clusterer
best_labels = best_clusterer.labels_

# Add cluster labels to DataFrame
df['cluster'] = best_labels

print(f"Using clustering algorithm: {best_clusterer.algorithm}")
print(f"Number of clusters: {len(set(best_labels))}")

In [ ]:
# Analyze cluster sizes
cluster_sizes = df['cluster'].value_counts().sort_index()
print("Cluster sizes:")
print(cluster_sizes)

# Plot
plt.figure(figsize=(10, 5))
cluster_sizes.plot(kind='bar')
plt.xlabel('Cluster')
plt.ylabel('Number of Rules')
plt.title('Rules per Cluster')
plt.show()

In [ ]:
# Sample rules from each cluster
msg_col = 'msg' if 'msg' in df.columns else 'message'

print("Sample rules from each cluster:\n")
for cluster_id in sorted(df['cluster'].unique()):
    if cluster_id == -1:
        print(f"Cluster {cluster_id} (NOISE):")
    else:
        print(f"Cluster {cluster_id}:")
    
    cluster_rules = df[df['cluster'] == cluster_id]
    sample_messages = cluster_rules[msg_col].head(3).tolist()
    
    for i, msg in enumerate(sample_messages, 1):
        print(f"  {i}. {msg}")
    print()

## 8. Save Results

In [ ]:
# Save DataFrame with cluster labels
df.to_pickle('../data/clustered_rules.pkl')
df.to_csv('../data/clustered_rules.csv', index=False)

# Save cluster labels separately
np.save('../data/cluster_labels.npy', best_labels)

print("Clustering results saved")

## 7. Generate Cluster Descriptions

Generate comprehensive descriptions for each cluster with automatic labeling, key terms, and representative rules.

In [ ]:
# Recreate feature extractor to get TF-IDF vocabulary
print("Recreating feature extractor for TF-IDF term extraction...")
extractor = features.RuleFeatureExtractor()
X_temp = extractor.create_feature_matrix(df, include_tfidf=True, tfidf_max_features=100)
print("Feature extractor ready")

In [ ]:
# Generate cluster descriptions
print("Generating cluster descriptions...")
descriptor = ClusterDescriptor(feature_extractor=extractor)
cluster_centers = best_clusterer.get_cluster_centers()
descriptions = descriptor.describe_all_clusters(X, best_labels, df, cluster_centers)
print(f"Generated descriptions for {len(descriptions)} clusters")

In [ ]:
# Display cluster descriptions (showing first 5 for brevity)
print("=" * 80)
print("CLUSTER DESCRIPTIONS")
print("=" * 80)
print()
for cluster_id in sorted(list(descriptions.keys())[:5]):
    print(format_cluster_description(descriptions[cluster_id]))
    print("\n")
print(f"\n... (showing 5 of {len(descriptions)} clusters)")
print("\nFor full descriptions, see outputs/cluster_descriptions.md or run 05_cluster_analysis.ipynb")

In [ ]:
# Export descriptions to files
print("Exporting cluster descriptions...")
descriptor.export_to_markdown(descriptions, '../outputs/cluster_descriptions.md')
descriptor.export_to_csv(descriptions, '../outputs/cluster_summary.csv')
print("\nCluster descriptions exported!")